In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# Parte 1

In [5]:
df=pd.read_excel("VillaVerde_WaterSystemData.xlsx", sheet_name="Datos")

FileNotFoundError: [Errno 2] No such file or directory: 'VillaVerde_WaterSystemData.xlsx'

In [ ]:
coords=pd.read_excel("VillaVerde_WaterSystemData.xlsx", sheet_name="Coordenadas")

In [ ]:
lmp=pd.read_excel("VillaVerde_WaterSystemData.xlsx", sheet_name="Limites")

In [ ]:
variables=['Punto','Caudal_Ls','Precip_mm_d','DBO5_mgL','DQO_mgL','SST_mgL','Ntot_mgL', 'Ptot_mgL', 'Campaña','TipoSistema']

In [ ]:
df=df[variables]

# Parte 2

In [ ]:
var_cols=['Caudal_Ls','Precip_mm_d','DBO5_mgL','DQO_mgL','SST_mgL','Ntot_mgL', 'Ptot_mgL']

In [ ]:
variable_sistema = (
    df
    .groupby(["TipoSistema", "Punto"])[var_cols]
    .agg(["min", "max", "mean", "std"])
)

variable_campa = (
    df
    .groupby(["Punto", "Campaña"])[var_cols]
    .agg(["min", "max", "mean", "std"])
)

writer = pd.ExcelWriter("Resultados_VillaVerde_Grupo01.xlsx")

variable_sistema.to_excel(writer, sheet_name="Variables Por Sistema_Punto")
variable_campa .to_excel(writer, sheet_name=" Variables por Campaña")

writer.close()

# Parte 3

In [ ]:
def graficar_variable_en_puntos(df, puntos, variable, limite, c, unidades):
    
    # ---- 1. Extraer LMP (si existe) ----
    tiene_limite = False
    if not limite.empty:
        lmp = float(limite.values[0])     # Valor numérico
        unidades = unidades.values[0]     # Unidades del LMP
        tiene_limite = True

    for punto in puntos:

        # Filtrar datos del punto
        p = df[df["Punto"] == punto][["Campaña", variable, "TipoSistema", "Punto"]]

        x = p["Campaña"]
        y = p[variable]

        plt.figure(figsize=(10,5))

        # Línea principal
        plt.plot(x, y, marker="o", linestyle="-", label="Medición", color=c)

        # ---- 3. LMP si existe ----
        if tiene_limite:

            # Detectar valores fuera del LMP (CORREGIDO)
            fuera = p[p[variable] > lmp]

            plt.scatter(
                fuera["Campaña"], fuera[variable],   # Cambiado Punto → Campaña
                s=120, facecolors='none', edgecolors='red',
                linewidth=2, label="Fuera del límite"
            )

            plt.axhline(
                lmp, linestyle="--", linewidth=2, color="red",
                label=f"LMP máximo = {lmp} {unidades}"
            )

        # ---- 4. Unidades por variable ----
        if variable == "Caudal_Ls":
            unidades = "L/s"
        elif variable == "Precip_mm_d":
            unidades = "mm/d"

        # ---- 5. Títulos y ejes ----
        plt.title(f"Mediciones de {variable} ({unidades}) en el punto {punto}")
        plt.xlabel("Campaña")
        plt.ylabel(f"{variable} ({unidades})")
        plt.ylim(0, 550)
        plt.grid(True)
        plt.legend()

        # ---- 6. Guardar figura ----
        plt.savefig(f"fig_{variable}_Campañas_{punto}.png", dpi=300)
        plt.close()


In [ ]:
puntos_rio=df[df["TipoSistema"]=="Potable"]["Punto"].unique()
variables_clave=['Caudal_Ls','Precip_mm_d']

for variable in variables_clave:
    unidad=lmp["Unidad"][(lmp["Variable"]==variable)]
    if lmp["LMP_max"][(lmp["Variable"]==variable) & (lmp["TipoSistema"]=="Rio")].empty==False:
        limite=lmp["LMP_max"][(lmp["Variable"]==variable) & (lmp["TipoSistema"]=="Rio")]
    else:
        limite=lmp["LMP_max"][(lmp["Variable"]==variable) & (lmp["TipoSistema"]=="Residual")]
    graficar_variable_en_puntos(df, puntos_rio, variable, limite,"blue",unidad)

In [ ]:
puntos_rio=df[df["TipoSistema"]=="Rio"]["Punto"].unique()
variables_clave=['Caudal_Ls','Precip_mm_d','DBO5_mgL','DQO_mgL','SST_mgL','Ntot_mgL', 'Ptot_mgL']

for variable in variables_clave:
    unidad=lmp["Unidad"][(lmp["Variable"]==variable)]
    if lmp["LMP_max"][(lmp["Variable"]==variable) & (lmp["TipoSistema"]=="Rio")].empty==False:
        limite=lmp["LMP_max"][(lmp["Variable"]==variable) & (lmp["TipoSistema"]=="Rio")]
    else:
        limite=lmp["LMP_max"][(lmp["Variable"]==variable) & (lmp["TipoSistema"]=="Residual")]
    graficar_variable_en_puntos(df, puntos_rio, variable, limite,"blue",unidad)

In [ ]:
puntos_residuales=df[df["TipoSistema"]=="Residual"]["Punto"].unique()
variables_clave=['Caudal_Ls','Precip_mm_d','DBO5_mgL','DQO_mgL','SST_mgL','Ntot_mgL', 'Ptot_mgL']

for variable in variables_clave:
    unidad=lmp["Unidad"][(lmp["Variable"]==variable)]
    limite=lmp["LMP_max"][(lmp["Variable"]==variable) & (lmp["TipoSistema"]=="Residual")]
    graficar_variable_en_puntos(df, puntos_residuales, variable, limite,"green",unidad)

In [ ]:


def graficar_variable_en_campañas(df, campañas, variable, limite, unidades):
    
    # ---- 1. Extraer LMP (si existe) ----
    tiene_limite = False
    if limite is not None and len(limite) > 0:
        LMP = float(limite.values[0])     # Valor numérico
        unidad_LMP = unidades.values[0]   # Unidades del LMP
        tiene_limite = True

    # ---- 2. Recorrer campañas ----
    for campaña in campañas:
        
        p = df[df["Campaña"] == campaña][["Campaña", variable, "TipoSistema", "Punto"]]

        x = p["Punto"]
        y = p[variable]

        plt.figure(figsize=(10, 5))

        # Línea principal
        plt.plot(x, y, marker="o", linestyle="-", label="Medición")

        # ---- 3. LMP si existe ----
        if tiene_limite:
            fuera = p[p[variable] > LMP]

            plt.scatter(
                fuera["Punto"], fuera[variable],
                s=120, facecolors='none', edgecolors='red',
                linewidth=2, label="Fuera del límite"
            )

            plt.axhline(
                LMP, linestyle="--", linewidth=2, color="red",
                label=f"LMP máximo = {LMP} {unidad_LMP}"
            )

        # ---- 4. Unidades por variable ----
        unidades_var = unidad_LMP if tiene_limite else ""

        if variable == "Caudal_Ls":
            unidades_var = "L/s"
        elif variable == "Precip_mm_d":
            unidades_var = "mm/d"

        # ---- 5. Títulos y etiquetas ----
        plt.title(f"Mediciones de {variable} ({unidades_var}) en campaña {campaña}",
                  fontsize=14)

        plt.xlabel("Punto", fontsize=12)
        plt.ylabel(f"{variable} ({unidades_var})", fontsize=12)
        plt.legend(fontsize=10)
        plt.grid(True, linestyle="--", alpha=0.4)

        # límites 
        plt.ylim(0, 550)

        # ---- 7. Guardar ----
        plt.savefig(f"fig_{variable}_Puntos_{campaña}.png",dpi=300)
        plt.close()

In [ ]:
campañas=df.Campaña.unique().tolist()
variable=['Caudal_Ls','Precip_mm_d','DBO5_mgL','DQO_mgL','SST_mgL','Ntot_mgL', 'Ptot_mgL']

for var in variable:
    unidad=lmp["Unidad"][(lmp["Variable"]==var)]
    limite=lmp["LMP_max"][(lmp["Variable"]==var) & (lmp["TipoSistema"]=="Residual")]
    graficar_variable_en_campañas(df, campañas, var, limite,unidad)

In [ ]:
def boxplot_por_punto(df, variable, limite, unidades):

    # ---- 1. Extraer LMP (si existe) ----
    tiene_limite = False
    if limite is not None and len(limite) > 0:
        LMP = float(limite.values[0])     # Valor numérico del LMP
        unidad_LMP = unidades.values[0]   # Unidad del LMP
        tiene_limite = True

    # ---- 2. Preparar unidades dinámicas ----
    unidades_var = unidad_LMP if tiene_limite else ""

    if variable == "Caudal_Ls":
        unidades_var = "L/s"
    elif variable == "Precip_mm_d":
        unidades_var = "mm/d"

    # ---- 3. Crear figura ----
    plt.figure(figsize=(10, 5))

    # Boxplot generalizado por punto
    df.boxplot(column=variable, by="Punto")

    # ---- 4. Línea del LMP si existe ----
    if tiene_limite:
        plt.axhline(
            LMP,
            linestyle="--",
            linewidth=2,
            color="red",
            label=f"LMP = {LMP} {unidad_LMP}"
        )

    # ---- 5. Títulos ----
    plt.title(f"{variable} por Punto", fontsize=14)
    plt.suptitle("")   # quita el título automático

    plt.xlabel("Punto", fontsize=12)
    plt.ylabel(f"{variable} ({unidades_var})", fontsize=12)

    # ---- 6. Límites y cuadrícula ----
    plt.ylim(0, 600)
    plt.grid(True, linestyle="--", alpha=0.4)

    plt.legend()

    # ---- 7. Guardar figura ----
    plt.savefig(f"boxplot_{variable}_por_punto.png", dpi=300)
    plt.close()


In [ ]:
variables=['Caudal_Ls','Precip_mm_d','DBO5_mgL','DQO_mgL','SST_mgL','Ntot_mgL', 'Ptot_mgL']
for var in variables:
    unidad=lmp["Unidad"][(lmp["Variable"]==var)]
    limite=lmp["LMP_max"][(lmp["Variable"]==var) & (lmp["TipoSistema"]=="Residual")]
    boxplot_por_punto(df, var, limite, unidad)

True

# Parte 4

NameError: name 'df' is not defined